In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io
import glob
from PIL import Image
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [2]:
import sys
sys.path.insert(1,'/kaggle/input/utils-file-to-train-and-validate-model')
import utils

##1.Upload the model

In [3]:
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights

weights = VGG16_Weights.DEFAULT
vgg_model = vgg16(weights=weights)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 227MB/s] 


##2.Freeze the model

In [38]:
vgg_model.requires_grad_(False)
next(iter(vgg_model.parameters())).requires_grad

False

##3.Add layers to model

In [33]:
vgg_model.classifier[0:3]

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
)

In [34]:
n_classes = 6
my_model = nn.Sequential(
    vgg_model.features,
    vgg_model.avgpool,
    nn.Flatten(),
    vgg_model.classifier[0:3],
    nn.Linear(4096,500),
    nn.ReLU(),
    nn.Linear(500,n_classes)
)
my_model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

##4.Compile the model

In [35]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(my_model.parameters())
my_model = torch.compile(my_model.to(device))

##5.Data augumentation

In [36]:
pre_trans = weights.transforms()
pre_trans

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [37]:
IMG_WIDTH, IMG_HEIGHT = (224,224)
random_trans = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(size=(IMG_WIDTH, IMG_HEIGHT),scale = (.7,1), ratio=(1,1)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.5, saturation=0.2, hue=0.2),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1))
    
])

##6.Dataloader, Dataset

In [20]:
DATA_LABELS = ["freshapples", "freshbanana", "freshoranges", "rottenapples", "rottenbanana", "rottenoranges"]

class MyDataset(Dataset):
    def __init__(self, data_dir):
        self.imgs = []
        self.labels = []

        for l_idx, label in enumerate(DATA_LABELS):
            data_paths = glob.glob(data_dir + label + '/*.png', recursive=True)
            for path in data_paths:
                img = tv_io.read_image(path, tv_io.ImageReadMode.RGB)
                self.imgs.append(pre_trans(img).to(device))
                self.labels.append(torch.tensor(l_idx).to(device))


    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.imgs)

In [21]:
n = 32
train_path = "/kaggle/input/fruits-dataset/fruits train/fruits train/train/"
train_data = MyDataset(train_path)
train_loader = DataLoader(train_data,batch_size = n,shuffle = True)
train_N = len(train_loader.dataset)

valid_path = "/kaggle/input/fruits-dataset/fruits valid/fruits valid/test/"
valid_data = MyDataset(valid_path)
valid_loader = DataLoader(valid_data,batch_size = n)
valid_N = len(valid_loader.dataset)

##7.Train model

In [39]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True
epochs = 20
for epoch in range(epochs):
    print('Epoch {}'.format(epoch))
    utils.train(my_model,train_loader,train_N,random_trans,optimizer,loss_function)
    utils.validate(my_model,valid_loader,valid_N,loss_function)

Epoch 0
Train - Loss: 75.5340 Accuracy: 0.9222
Valid - Loss: 26.8933 Accuracy: 0.9181
Epoch 1
Train - Loss: 49.1373 Accuracy: 0.9469
Valid - Loss: 25.8245 Accuracy: 0.9259
Epoch 2
Train - Loss: 44.1986 Accuracy: 0.9537
Valid - Loss: 25.9483 Accuracy: 0.9311
Epoch 3
Train - Loss: 40.9130 Accuracy: 0.9574
Valid - Loss: 26.2410 Accuracy: 0.9288
Epoch 4
Train - Loss: 36.6116 Accuracy: 0.9624
Valid - Loss: 23.5405 Accuracy: 0.9370
Epoch 5
Train - Loss: 34.3469 Accuracy: 0.9653
Valid - Loss: 19.7714 Accuracy: 0.9455
Epoch 6
Train - Loss: 35.2255 Accuracy: 0.9641
Valid - Loss: 16.2316 Accuracy: 0.9496
Epoch 7
Train - Loss: 31.4263 Accuracy: 0.9660
Valid - Loss: 47.2057 Accuracy: 0.8984
Epoch 8
Train - Loss: 30.1661 Accuracy: 0.9690
Valid - Loss: 20.0555 Accuracy: 0.9444
Epoch 9
Train - Loss: 32.0315 Accuracy: 0.9683
Valid - Loss: 27.3487 Accuracy: 0.9303
Epoch 10
Train - Loss: 31.2393 Accuracy: 0.9693
Valid - Loss: 23.2281 Accuracy: 0.9362
Epoch 11
Train - Loss: 26.3893 Accuracy: 0.9737
Valid

##8.Fine-tuning

In [40]:
vgg_model.requires_grad_(True)
optimizer = Adam(my_model.parameters(),lr =.000001)

In [41]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True
epochs = 5
for epoch in range(epochs):
  print('Epoch {}'.format(epoch))
  utils.train(my_model,train_loader,train_N,random_trans,optimizer,loss_function)
  utils.validate(my_model,valid_loader,valid_N,loss_function)

Epoch 0
Train - Loss: 21.1143 Accuracy: 0.9779
Valid - Loss: 23.9098 Accuracy: 0.9329
Epoch 1
Train - Loss: 19.9447 Accuracy: 0.9804
Valid - Loss: 26.5417 Accuracy: 0.9285
Epoch 2
Train - Loss: 18.2132 Accuracy: 0.9814
Valid - Loss: 20.8646 Accuracy: 0.9392
Epoch 3
Train - Loss: 17.4566 Accuracy: 0.9833
Valid - Loss: 21.3377 Accuracy: 0.9370
Epoch 4
Train - Loss: 17.3893 Accuracy: 0.9836
Valid - Loss: 21.8032 Accuracy: 0.9407


##9.Evaluate the model

In [42]:

utils.validate(my_model, valid_loader, valid_N, loss_function)

Valid - Loss: 21.8032 Accuracy: 0.9407
